# ABS Quarterly National Accounts 5206

## Python set-up

In [1]:
# system imports
import re
import sys

# analytic imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from abs_data_capture import (
    find_id,
    fix_abs_title,
    get_ABS_meta_and_data,
    get_fs_constants,
    get_identifier,
    get_meta_constants,
    get_plot_constants,
)

# local imports
from henderson import hma
from plotting import (
    calc_growth,
    clear_chart_dir,
    finalise_plot,
    line_plot,
    plot_covid_recovery,
    plot_growth_finalise,
    recalibrate_series,
    set_chart_dir,
)

# pandas display settings
pd.options.display.max_rows = None
pd.options.display.max_columns = 999

# display charts within this notebook
SHOW = False

## Get data from ABS

In [2]:
catalogue_id = "5206"
source, CHART_DIR, META_DATA = get_fs_constants(catalogue_id)

abs_dict = get_ABS_meta_and_data(catalogue_id)
if abs_dict is None:
    sys.exit(-1)

meta = abs_dict[META_DATA]
RECENT, plot_times, plot_tags = get_plot_constants(meta)
did_col, id_col, table_col, type_col, unit_col = get_meta_constants()

Found URL for a ZIP file on ABS web page
Retrieving zip-file from cache ...
Extracting DataFrames from the zip-file ...


In [3]:
# extract meta data, etc.
CVM = "Chain volume measures"
value_text = "$ Millions"

## Plot

### Plot setup

In [4]:
plt.style.use("fivethirtyeight")
clear_chart_dir(CHART_DIR)
set_chart_dir(CHART_DIR)

### Growth charts - based on seasonally adjusted series

In [5]:
# identify available headline series from metadata
table = "1"
data = abs_dict[table]

series_type = "Seasonally Adjusted"
rows = meta[
    (meta[table_col] == table)
    & (meta[type_col] == series_type)
    & (meta[unit_col].isin([value_text, "$"]))
    & (meta[did_col].str.contains(CVM))
]

for index, row in rows.iterrows():
    # identify data
    series_id, units, title = row[id_col], row[unit_col], row[did_col]
    series, units = recalibrate_series(data[series_id], units, verbose=True)
    units = f"{units} / Quarter"  # make time frame explicit
    lfooter = f"Australia. {series_type}. "
    title, lfooter = fix_abs_title(title, lfooter)
    series.name = title.replace(":", "").strip()

    common = {
        "rfooter": f"{source} {table}",
        "lfooter": lfooter,
    }

    # plot
    line_plot(
        series,
        dropna=True,
        starts=plot_times,
        tags=plot_tags,
        title=title,
        ylabel=units,
        pre_tag="raw-charts-",
        show=SHOW,
        **common,
    )

    plot_covid_recovery(
        series,
        title=title,
        ylabel=f"{units}",
        tags="covid-qtr",
        rfooter=f"{source} {table}",
        lfooter=lfooter,
        show=SHOW,
    )

    growth = calc_growth(series)
    print(f"Last {title}: ", [g.iloc[-1] for g in growth])
    for start, tag in zip(plot_times, plot_tags):
        plot_growth_finalise(
            *growth,
            from_=start,
            tag=tag,
            title=f"{title} growth",
            pre_tag="growth-charts-",
            show=SHOW and start is not None,
            **common,
        )

Last Gross domestic product::  [2.053587478223484, 0.21197974232500805]
recalibrate() is wrking with money.
Last GDP per capita::  [-0.34270650263621416, -0.46515710022819734]
Last Gross value added market sector::  [1.8764308797656115, -0.07533292096781885]
Last Net domestic product::  [1.9298401996693748, 0.11540864609740353]
Last Real gross domestic income::  [-0.4388774504966664, -0.45319620385467774]
Last Real gross national income::  [1.2335395982939978, -0.34824542548692605]
Last Real net national disposable income::  [0.8760462215810838, -0.6410744339854113]
recalibrate() is wrking with money.
Last Real net national disposable income per capita::  [-1.4933752526386845, -1.3105349007255795]


### Long run annual GDP Growth - using a Henderson moving average

In [6]:
def chart_gdp_long_run_growth(chart: str) -> None:
    # key data sources
    table = "1"
    data = abs_dict[table]
    series_type = "Seasonally Adjusted"

    # select series
    selector = {
        table: table_col,
        chart: did_col,
        series_type: type_col,
        CVM: did_col,
        "$": unit_col,
    }
    id, units = find_id(meta, selector, verbose=False)

    # plot original
    ax = None  # so we can comment out a plot code-line
    q_gdp = data[id].dropna()
    series = (q_gdp.pct_change(periods=4, fill_method=None) * 100).dropna()
    ax = series.plot(color="darkblue", lw=0.5, ls="-", ax=ax, label="Annual growth")

    # plot Henderson Moving Average
    HENDERSON_N = 13  # months ==> multi-year Henderson moving average
    smoothed = hma(series, HENDERSON_N)
    ax = smoothed.plot(
        color="darkorange",
        lw=1.5,
        ls="-",
        ax=ax,
        label=f"{HENDERSON_N}-term Henderson moving average",
    )

    # plot decadal averages
    decades = series.index.year.astype(str).str[2:3].unique()
    label = "Decadal mean (DM) annual growth"
    for decade in decades:
        d_series = series[series.index.year.astype(str).str[2:3] == decade]
        d_series = pd.Series(
            np.repeat(d_series.mean(), len(d_series)), index=d_series.index
        )
        ax = d_series.plot(color="darkred", lw=2, ls="--", ax=ax, label=label)
        label = None
        ax.text(
            x=d_series.index[-1],
            y=series.min(),
            s=f"DM = {d_series.iloc[-1]:0.2f}%",
            rotation=90,
            ha="center",
            size="x-small",
        )

    # plot compound annual growth rate over a decade
    YEARS = 10
    PERIODS_PER_YEAR = 4
    annual = (
        ((q_gdp / q_gdp.shift(YEARS * PERIODS_PER_YEAR)) ** (1 / YEARS)) - 1
    ) * 100
    ax = annual.plot(
        color="dodgerblue",
        lw=3,
        ls="-",
        ax=ax,
        label="Annual compound growth over decade",
    )

    # finalise the plot
    ax.legend(loc="best", prop={"size": "x-small"})
    title = f"YoY Growth - {chart} - Long-run trends"
    finalise_plot(
        ax,
        title=title,
        ylabel="Per cent / year",
        rfooter=f"{source} {table}",
        lfooter=f"Australia. {series_type.capitalize()}. "
        f"{CVM}. Compared with the same quarter in the previous year.",
        pre_tag="long-run-growth-",
        y0=True,
        show=SHOW,
    )

In [7]:
charts = [
    "Gross domestic product",
    "GDP per capita",
]
for chart in charts:
    chart_gdp_long_run_growth(chart)

### Four quarter rolling totals - original series

In [8]:
table = "1"
data = abs_dict[table]
series_type = "Original"

charts = [
    "Gross domestic product",
    "GDP per capita",
]
for chart in charts:
    selector = {
        table: table_col,
        chart: did_col,
        series_type: type_col,
        CVM: did_col,
        "$": unit_col,
    }
    id, units = find_id(meta, selector, verbose=False)
    rolling4Q = data[id].rolling(4).sum()
    # units = "$ Number" if units == "$" else units
    rolling4Q, units = recalibrate_series(rolling4Q, units)
    rolling4Q.name = chart

    title = meta.loc[meta[id_col] == id, did_col].iloc[0]
    lfooter = f"Australia. 4Q rolling sum. {series_type} series. "
    title, lfooter = fix_abs_title(title, lfooter)
    plot_covid_recovery(
        rolling4Q,
        title=title,
        ylabel=f"{units} / year",
        tags="covid-annual",
        rfooter=f"{source} {table}",
        lfooter=lfooter,
        pre_tag="gdp-",
        show=SHOW,
    )

### Implicit population estimates

In [9]:
table = "1"
data = abs_dict[table]
series_type = "Seasonally Adjusted"

pairs = {
    # $                                         $ Millions
    (
        "GDP per capita: Chain volume measures ;",
        "Gross domestic product: Chain volume measures ;",
    ),
    ("GDP per capita: Current prices ;", "Gross domestic product: Current prices ;"),
}

pop_series = []
for pair in pairs:
    data_items = []
    for index, series in enumerate(pair):
        selector = {
            table: table_col,
            series_type: type_col,
            series: did_col,
        }
        id, units = find_id(meta, selector, verbose=True)
        data_items.append(data[id])
    series = data_items[1] / data_items[0]
    pop_series.append(series)
pop_series = pd.DataFrame(pop_series).T.mean(axis="columns").dropna()
pop_series.name = "Australian population"

title = "Implicit population from ABS National Accounts"
common = {
    "rfooter": f"{source} {table}",
    "lfooter": "Australia. Calculated. ",
    "show": SHOW,
}
plot_covid_recovery(
    pop_series,
    title=title,
    ylabel=f"Millions",
    tags="covid",
    **common,
)

growth = calc_growth(pop_series)
growth2 = []
growth2.append(pop_series.diff(periods=4) * 1000)
growth2.append(pop_series.diff(periods=1) * 1000)

print(f"Last {title}: ", [g.iloc[-1] for g in growth])
new_title = title.replace("population", "pop. growth")
for start, tag in zip(plot_times, plot_tags):
    plot_growth_finalise(
        *growth,
        from_=start,
        tag=tag,
        title=new_title,
        pre_tag="population-percent-",
        **common,
    )
    plot_growth_finalise(
        *growth2,
        from_=start,
        tag=tag,
        ylabel="Thousands",
        title=new_title,
        pre_tag="population-change-",
        **common,
    )

Last Implicit population from ABS National Accounts:  [2.4055291661836176, 0.6803765234222103]


### Implicit price deflators

In [10]:
table = "5"
data = abs_dict[table]

keys = {
    "GDP": "GROSS DOMESTIC PRODUCT ;",
    "HFCE": "Households ;  Final consumption expenditure ;",
}

deflators = {}  # used in calculations below ...
for prefix, key in keys.items():
    ident = meta[(meta[did_col] == key) & (meta[table_col] == table)][id_col].iloc[0]
    series = data[ident].dropna()
    print(series.tail())
    deflators[prefix] = series / series.iloc[-1]  # rebase
    growth = series.pct_change(4, fill_method=None) * 100
    title = f"Growth in the {prefix} Implicit Prices"

    line_plot(
        growth,
        title=title,
        ylabel="Growth: Per cent per year",
        rfooter=f"{source} {table}",
        lfooter="Calculated from the seasonally adjusted index",
        y0=True,
        pre_tag="deflators-",
        show=SHOW,
    )

Series ID
2022Q3    105.3
2022Q4    106.1
2023Q1    108.0
2023Q2    106.7
2023Q3    107.8
Freq: Q-DEC, Name: A2303730T, dtype: float64
Series ID
2022Q3    104.3
2022Q4    106.1
2023Q1    107.2
2023Q2    108.6
2023Q3    110.1
Freq: Q-DEC, Name: A2303940R, dtype: float64


#### Check deflator accuracy

In [11]:
table = "5"
data = abs_dict[table]
row = meta[
    (meta[table_col] == table)
    & meta[did_col].str.contains("[Pp]ercent")
    & meta[did_col].str.contains("GROSS DOMESTIC PRODUCT")
]
id = row[id_col].values[0]
series = (data[id] / 100 + 1).cumprod()
series.iloc[0] = 1.0
series = series / series.iloc[-1]
(series - deflators["GDP"]).mean()

-4.71206634191359e-05

### Savings - current prices and GDP-deflator adjusted

In [12]:
table = "11"
data = abs_dict[table]
series_type = "Seasonally Adjusted"
series_focus = "Net saving"

headline_plots = meta[
    (meta[table_col] == table)
    & (meta[type_col] == series_type)
    & (meta[did_col].str.contains(series_focus))
][did_col]

for description in headline_plots:
    # identify data
    series, units = get_identifier(meta, description, series_type, table)
    units = f"{units} / Qtr"
    plotable, units = recalibrate_series(data[series].dropna(), units)
    title = description.replace(" ;", "").replace("  ", " ").capitalize()

    common = {
        "title": title,
        "y0": True,
        "rfooter": f"{source} {table}",
        "pre_tag": "saving-",
        "show": SHOW,
    }
    lfooter = f"Australia. {series_type.capitalize()} series. Current prices. "

    line_plot(
        plotable,
        starts=plot_times,
        tags=plot_tags,
        lfooter=lfooter,
        ylabel=units,
        **common,
    )

    for deflator in "GDP", "HFCE":
        plotable = (plotable / deflators[deflator]).dropna()
        plotable.name = "Series"
        plot_covid_recovery(
            plotable,
            tags=f"covid-{deflator}",
            lfooter=f"{lfooter}Prices adjusted by {deflator} deflator. ",
            ylabel=f"{units} (inflation adj)",
            **common,
        )

### Gross Value Added (GVA) by Industry

In [13]:
# identify available series from metadata
table = "6"
data = abs_dict[table]

series_type = "Seasonally Adjusted"
industries = meta[
    (meta[table_col] == table)
    & (meta[type_col] == series_type)
    & (meta[unit_col] == value_text)  # limit to money
][did_col]
industry_start = pd.Period("1974-09-01", freq="Q")

for industry in industries:
    # get data
    selector = {
        table: table_col,
        series_type: type_col,
        industry: did_col,
    }

    id, units = find_id(meta, selector, exact=True, verbose=False)
    units = f"{units}  / Quarter"
    series, units = recalibrate_series(data[id].dropna(), units)

    title = industry.replace(" ;", "").strip()
    if not title.endswith(")"):
        title = re.sub(r"^[^\(]+\(", "(", title)
    title = f" GVA: {title}"

    line_plot(
        series,
        starts=plot_times,
        tags=plot_tags,
        title=title,
        ylabel=units,
        rfooter=f"{source} {table}",
        lfooter=(f"{series_type.capitalize()}, " f"{CVM.lower()}"),
        pre_tag="gva-",
        show=SHOW,
    )

###  Household Final Consumption Expenditure

In [14]:
# identify available series from metadata
table = "8"
data = abs_dict[table]
series_type = "Seasonally Adjusted"
hfce = meta[
    (meta[table_col] == table)
    & (meta[type_col] == series_type)
    & (meta[unit_col] == "$ Millions")
    & (meta[did_col].str.contains(CVM))  # limit to money
][did_col]

for description in hfce:
    print(description)
    # get data
    series, units = get_identifier(meta, description, series_type, table)
    units = f"{units} / Quarter"
    plotable, units = recalibrate_series(data[series].dropna(), units)
    plotable.name = f"{series_type.capitalize()} series"
    title = description.replace(f": {CVM} ;", "")
    separator = "\n" if len(title) > 25 else " "
    title = f"Household consumption:{separator}{title}"

    common = {
        "title": title,
        "ylabel": units,
        "rfooter": f"{source} {table}",
        "lfooter": f"Australia. {CVM.capitalize()} ",
        "pre_tag": "hfce-",
        "show": SHOW,
    }

    # plot - complete series - and recent showing covid recoverhy
    line_plot(
        plotable,
        **common,
    )

    plot_covid_recovery(
        plotable,
        tags="covid",
        **common,
    )

    # remember HFCE for next cell ...
    if "FINAL CONSUMPTION EXPENDITURE" in description:
        HFCE_q_series = plotable
        HFCE_q_units = units

Food: Chain volume measures ;
Cigarettes and tobacco: Chain volume measures ;
Alcoholic beverages: Chain volume measures ;
Clothing and footwear: Chain volume measures ;
Rent and other dwelling services: Chain volume measures ;
Electricity, gas and other fuel: Chain volume measures ;
Furnishings and household equipment: Chain volume measures ;
Health: Chain volume measures ;
Purchase of vehicles: Chain volume measures ;
Operation of vehicles: Chain volume measures ;
Transport services: Chain volume measures ;
Communications: Chain volume measures ;
Recreation and culture: Chain volume measures ;
Education services: Chain volume measures ;
Hotels, cafes and restaurants: Chain volume measures ;
Insurance and other financial services: Chain volume measures ;
Other goods and services: Chain volume measures ;
FINAL CONSUMPTION EXPENDITURE: Chain volume measures ;


In [15]:
table = "1"
data = abs_dict[table]
series_type = "Seasonally Adjusted"
chart = "Gross domestic product"

selector = {
    table: table_col,
    chart: did_col,
    series_type: type_col,
    CVM: did_col,
    "$": unit_col,
}
id, units = find_id(meta, selector, verbose=False)
series = data[id]
series, units = recalibrate_series(series, units)
GDP_q_series = series
GDP_q_units = units

In [16]:
HFCE_proportion = HFCE_q_series / GDP_q_series * 100
line_plot(
    HFCE_proportion,
    title="HFCE as a proportion of GDP",
    ylabel="Per cent",
    rfooter=f"{source} {table}",
    lfooter=(f"Seasonally adjusted series. {CVM.lower()}"),
    pre_tag="hfce-",
    show=SHOW,
)
print(HFCE_proportion.mean())

49.22915116671011


### Taxes

In [17]:
# identify available series from metadata
table = "22"
data = abs_dict[table]
series_type = "Seasonally Adjusted"
tax_plots = meta[(meta[table_col] == table) & (meta[type_col] == series_type)][did_col]

for plot in tax_plots:
    # data capture
    ident, units = get_identifier(meta, plot, series_type, table)
    units = f"{units} / Quarter"
    series, units = recalibrate_series(data[ident], units)

    # plot
    title = plot.replace(" ;", "")
    line_plot(
        series,
        starts=RECENT,
        title=title,
        ylabel=units,
        rfooter=f"{source} {table}",
        lfooter=f"Australia. {series_type.capitalize()} series. Current Prices.",
        pre_tag="taxes-",
        show=SHOW,
    )

In [18]:
# taxes on income divided by income


### Government Benefits Payments

In [19]:
table = "23"
data = abs_dict[table]
series_type = "Original"

plots_payments = meta[(meta[table_col] == table) & (meta[type_col] == series_type)][
    did_col
]

for plot in plots_payments:
    # get data
    ident, units = get_identifier(meta, plot, series_type, table)
    units = f"{units} / Quarter"
    series = data[ident].dropna()
    series, units = recalibrate_series(series, units)

    # plot
    title = plot.replace("General government - National ;  ", "Federal Govt - ")
    title = title.replace(
        "General government - State and local ;  ", "State or Local Govt - "
    )
    title = title.replace("General government ;  ", "All Govt - ")
    title = title.replace(" ;", "")
    line_plot(
        series,
        starts=RECENT,
        title=title,
        pre_tag="payments-",
        ylabel=f"{units}",
        rfooter=f"{source} {table}",
        lfooter=f"Australia. {series_type.capitalize()} series. Current prices.",
        show=SHOW,
    )

### Total Wages v Total Profit

In [20]:
wages_table = "7"
series_type = "Seasonally Adjusted"  # 'Original'
wages_id, wage_units = find_id(
    meta,
    {
        wages_table: table_col,
        series_type: type_col,
        "Compensation of employees ;": did_col,
    },
    exact=True,
    verbose=False,
)
wages = abs_dict[wages_table][wages_id]

income_table = "7"
tf_income_id, tf_income_units = find_id(
    meta,
    {
        income_table: table_col,
        series_type: type_col,
        "Total factor income ;": did_col,
    },
    exact=True,
    verbose=False,
)
tf_income = abs_dict[income_table][tf_income_id]

assert wage_units == tf_income_units

profit_share_table = "24"
profit_ratio_id, share_units = find_id(
    meta,
    {
        profit_share_table: table_col,
        "Seasonally Adjusted": type_col,
        "Profits share of total factor income: Ratio ;": did_col,
    },
    exact=True,
    verbose=False,
)
profit_tfi_share = abs_dict[profit_share_table][profit_ratio_id] / 100
profits = profit_tfi_share * tf_income

# plot shares
assert wage_units == tf_income_units
profit_wage_share = profits / (profits + wages) * 100
hma_term = 7
trend = hma(profit_wage_share, hma_term)
df = pd.DataFrame(
    {
        "Profit share": profit_wage_share,
        "Henderson moving avergae": trend,
    }
)
line_plot(
    df,
    width=[1, 3],
    title="Profits as a share of profits plus wages",
    ylabel="Per cent",
    rfooter=f"{source} Tables: {wages_table}, {profit_share_table}",
    lfooter=f"Australia. {series_type.capitalize()} series. "
    f"{hma_term}-term Henderson moving average. ",
    pre_tag="profits-",
    show=SHOW,
)

# plot values
assert "Millions" in wage_units
df = pd.DataFrame(
    {
        "Wages": wages / 1000,
        "Profits": profits / 1000,
    }
)
line_plot(
    df,
    title="Profits vs Wages",
    ylabel="$ Billions",
    rfooter=f"{source} Tables: {wages_table}, {profit_share_table}",
    lfooter=f"Australia. {series_type.capitalize()} series. Current Prices. ",
    pre_tag="profits-",
    show=SHOW,
)

# plot index
assert wages.index[0] == profits.index[0]
df = pd.DataFrame(
    {
        "Wages": wages / wages.iloc[0],
        "Profits": profits / profits.iloc[0],
    }
)
line_plot(
    df,
    title="Profits index vs Wages index",
    ylabel=f"Index ({wages.index[0]} = 1)",
    rfooter=f"{source} Tables: {wages_table}, {profit_share_table}",
    lfooter=f"Australia. {series_type.capitalize()} series. Current Prices. ",
    pre_tag="profits-",
    show=SHOW,
)

### Real Wages (using implicit price deflator)

In [21]:
table = "24"
data = abs_dict[table]
series_type = "Seasonally Adjusted"
dids = [
    "Average compensation per employee: Current prices ;",
    "Compensation of employees per hour: Current prices ;",
]

dates = None, pd.Period("2019-12-01", freq="Q")
tags = "", "post-COVID"

for did in dids:
    id, units = get_identifier(meta, did, series_type, table, False)
    series = (data[id] / deflators["HFCE"]).dropna()
    title = did.split(":")[0].strip()
    series.name = title

    common = {
        "title": f"Real {title}",
        "ylabel": f"{units} (inflation adjusted)",
        "rfooter": f"{source} {table}",
        "lfooter": f"Australia. {series_type.capitalize()} series. "
        "Prices adjusted by HFCE deflator. ",
        "pre_tag": "wages-",
        "show": SHOW,
    }

    line_plot(
        series,
        **common,
    )

    plot_covid_recovery(
        series,
        tags="covid",
        **common,
    )

### GDP expenditure compenents

The sum of final consumption spending in the economy: GDP(E) = C + I + G + (X - M)
* C = (private) consumption
* I = business inestor spending on captial goods plus change in inventories
* G = government consumption spending (not tax transfers, not wages)
* (X - M) = net exports = exports - imports

Note - there is a lot of uncertainty with the pre-2000 ABS data - see add checks below, but fundamentally, the components typically add to less than GDP(E). The biggest disparity is between GNE+X-M and GDP. Further complicated in that the change in inventories data is only available from 1974. 

Note the GDP deflators are calculated from the expenditure approach.

In [22]:
table = "2"
series_type = "Seasonally Adjusted"
if SHOW:
    display(
        meta[
            (meta[table_col] == table)
            & meta[unit_col].str.contains("$ Million", regex=False)
            & (meta[type_col] == series_type)
        ]
    )

In [23]:
def plot_gdp_series(
    series,
    units,
    series_name,
    rfooter,
    lfooter,
    denominators,
    pre_tag="GDP(E)-",
    show=SHOW,
):
    line_plot(
        series,
        starts=RECENT,
        title=series_name,
        pre_tag=pre_tag,
        ylabel=units,
        rfooter=rfooter,
        lfooter=lfooter,
        y0=True,
        show=show,
    )

    for label, d_series in denominators.items():
        p_series = series / d_series * 100
        line_plot(
            p_series,
            starts="2010-01-01",
            title=series_name,
            pre_tag=pre_tag,
            tags=f"percent-{label}",
            ylabel=f"Per cent of {label}",
            rfooter=rfooter,
            lfooter=lfooter,
            y0=True,
            show=show,
        )

In [24]:
table = "2"
data = abs_dict[table]
series_type = "Seasonally Adjusted"

denominators = {
    "GDP": "GROSS DOMESTIC PRODUCT ;",
    "GNE": "Gross national expenditure ;",
}

for denominator, denom_did in denominators.items():
    id, units = get_identifier(meta, denom_did, series_type, table)
    series = data[id] / 1000
    units = units.replace("Millions", "Billions")
    denominators[denominator] = series

numerators = {
    "C": "Households ;  Final consumption expenditure ;",
    "Invest": "All sectors ;  Gross fixed capital formation ;",
    "Invent": "Changes in inventories ;",
    "G": "General government ;  Final consumption expenditure ;",
    "S": "Statistical discrepancy (E) ;",
    "X": "Exports of goods and services ;",
    "M": "Imports of goods and services ;",  # must be last, because we subtract it
}

gdp_exp_components = pd.DataFrame()
for component, did in numerators.items():
    id, units = get_identifier(meta, did, series_type, table)
    series = data[id] / 1000
    units = units.replace("Millions", "Billions")

    gdp_exp_components[component] = series
    rfooter = f"{source} {table}"
    lfooter = f"Australia. {series_type.capitalize()} series. {CVM}."
    plot_gdp_series(
        series, units, did.replace(" ;", ""), rfooter, lfooter, denominators
    )

# let's look at cumulative adding-errors in GDP(E)
gdp = (
    gdp_exp_components[gdp_exp_components.columns[:-1]].sum(axis=1)
    - gdp_exp_components[gdp_exp_components.columns[-1]]
)
plot_gdp_series(
    gdp, units, "GDP(E) from summed components", rfooter, lfooter, denominators, False
)

#### Further exploration of GDP(E) adding errors

In [25]:
def add_check(did_list):
    repository = {}
    for did in did_list:
        id, units = get_identifier(meta, did, series_type, table)
        series = data[id]
        repository[did] = series
    repo_df = pd.DataFrame(repository)
    analysis = (
        repo_df[repo_df.columns[:-1]].sum(axis=1) / repo_df[repo_df.columns[-1]] * 100
    )
    mean = analysis.mean()
    print(mean)
    return repo_df, analysis

In [26]:
fce = [
    "General government ;  Final consumption expenditure ;",
    "Households ;  Final consumption expenditure ;",
    "All sectors ;  Final consumption expenditure ;",
]
df, analysis = add_check(fce)

99.86088091387394


In [27]:
dfd = [
    "All sectors ;  Final consumption expenditure ;",
    "All sectors ;  Gross fixed capital formation ;",
    "Domestic final demand ;",
]
df, analysis = add_check(dfd)

100.47480800155746


In [28]:
# Note - problem here is that changes in inventories was
#        not recordsed prior to 1974.

gne = [
    "Domestic final demand ;",
    "Changes in inventories ;",
    "Gross national expenditure ;",
]
_df, analysis = add_check(gne)

98.18439617662712


## Finished

In [29]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Fri Dec 15 2023 17:44:48

Python implementation: CPython
Python version       : 3.11.6
IPython version      : 8.18.1

matplotlib: 3.8.2
sys       : 3.11.6 | packaged by conda-forge | (main, Oct  3 2023, 10:37:07) [Clang 15.0.7 ]
re        : 2.2.1
pandas    : 2.1.3
numpy     : 1.26.2

Watermark: 2.4.3



In [30]:
print("Finished")

Finished
